# **Variational Auto-Encoder**

In [ ]:
import gc
import psutil
import multiprocessing as mp
import copy
mp.cpu_count()
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras import Input,Sequential, initializers, optimizers, callbacks, layers, models
from keras.models import Model,Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,BatchNormalization,Lambda,Activation,Input,Flatten,Reshape,Conv2DTranspose
import keras.backend as K
from keras.layers.merge import add
from sklearn.model_selection import train_test_split
import os
import glob
from time import time,asctime
from random import randint as r
import random

In [ ]:
imgs = glob.glob("../input/celeba-dataset/img_align_celeba/img_align_celeba/*.jpg")
imgs[0]

# Trai data 

In [ ]:
train_y = []
train_y2 = []
for _ in range(0,100000):
  if _%20000 == 0:
    print("{} / 100000".format(_))
  img = cv2.imread(imgs[_])
  img = cv2.resize(img,(32,32))
  train_y.append(img.astype("float32")/255.0)
for _ in range(100000,200000):
  if _%20000 == 0:
    print("{} / 200000".format(_))
  img = cv2.imread(imgs[_])
  img = cv2.resize(img,(32,32))
  train_y2.append(img.astype("float32")/255.0)
train_y = np.array(train_y)
train_y2 = np.array(train_y2)
Y_data = np.vstack((train_y,train_y2))
print(psutil.virtual_memory())
del train_y,train_y2
gc.collect()
print(psutil.virtual_memory())

# Test data

In [ ]:
test_Y = []
for _ in range(200000,202599):
    if _%2000 == 0:
        print("{} / 202599".format(_))
    img = cv2.imread(imgs[_])
    img = cv2.resize(img,(32,32))
    test_Y.append(img.astype("float32")/255.0)
test_Y = np.array(test_Y)

# Sampler function
**Variational Auto Encoder has a sampler In VAE, the encoder outputs two vectors.one is mean and the other is standard_deviation.sample from these two are taken as a final vector that can be done using the sampler function.**

In [ ]:
def sample_z(layers):
    std_norm = K.random_normal(shape=(K.shape(layers[0])[0], latent_dim), mean=0, stddev=1)
    return layers[0] + layers[1]*std_norm

# Encoder part

In [ ]:
latent_dim = 50
total_epoch = 50
batch_size = 32
img_dim = (32, 32)

encoder_inputs = keras.Input(shape=(*img_dim, 3))

x = layers.Conv2D(32, kernel_size=(5,5), padding='SAME')(encoder_inputs)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(0.2)(x)

x = layers.Conv2D(32, kernel_size=(5,5), strides=(2,2),padding='SAME')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(0.2)(x)

x = layers.Conv2D(64, kernel_size=(5,5), strides=(2,2),padding='SAME')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(0.2)(x)

x = layers.Conv2D(64, kernel_size=(5,5), strides=(1,1), padding='SAME')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(0.2)(x)

x = layers.Conv2D(128, kernel_size=(5,5), strides=(1,1), padding='SAME')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(0.2)(x)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation="relu")(x)

x = layers.Dense(64, activation="relu")(x)

mu = layers.Dense(latent_dim)(x)
log_sigma = layers.Dense(latent_dim)(x)

z = layers.Lambda(sample_z)([mu, log_sigma])

encoder = keras.Model(encoder_inputs,z)

encoder.summary()


# Decoder part

In [ ]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(8 * 8 * 128, activation="relu")(latent_inputs)
x = layers.Reshape((8, 8, 128))(x)

x = layers.Conv2D(128, kernel_size=(5,5), strides=(1,1), padding='SAME')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(0.2)(x)

x = layers.Conv2DTranspose(64, kernel_size=(5,5),strides=(1,1), padding='SAME')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Conv2DTranspose(64, kernel_size=(5,5),strides=(2,2), padding='SAME')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(0.2)(x)

x = layers.Conv2DTranspose(32, kernel_size=(5,5),strides=(2,2),padding='SAME')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(0.2)(x)

x = layers.Conv2DTranspose(32, kernel_size=(5,5), padding='SAME')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(0.2)(x)
decoder_outputs = layers.Conv2DTranspose(3, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs)
decoder.summary()

# LOSS FUNCTIONS

In [ ]:
def reconstruction_loss(y_true, y_pred):
    return K.mean(K.square(y_true - y_pred))

def kl_loss(y_true, y_pred):
    kl_loss = K.abs(keras.losses.KLD(y_true, y_pred))
    return kl_loss

def vae_loss(y_true, y_pred):
    return reconstruction_loss(y_true, y_pred)+0.25*kl_loss(y_true, y_pred)

# Connecting the Encoder and Decoder to make the **Auto Encoder**

In [ ]:
vae = keras.Model(encoder_inputs, decoder(encoder(encoder_inputs)))

In [ ]:
vae.compile(optimizer = "adam",loss = vae_loss,metrics = [kl_loss,reconstruction_loss])

# Training the VAE

In [ ]:
vae.fit(Y_data,Y_data,batch_size = 32,epochs = total_epoch)

# Displaying result

In [ ]:
pred = vae.predict(test_Y)

In [ ]:
temp = r(0,2599)
print(temp)
plt.subplot(1,2,1)
plt.imshow(test_Y[temp])
plt.subplot(1,2,2)
plt.imshow(pred[temp])

Generating a new face by passing a random normal sample of size (32,32,3) and observing the output

In [ ]:
gen =np.random.uniform(0, 1, size = (1,32,32,3))
gen_sample = vae.predict(gen)
plt.subplot(1,2,1)
plt.imshow(gen[0])
plt.subplot(1,2,2)
plt.imshow(gen_sample[0])

In [ ]:
img_l=[]
img_path =("../input/someface/14997843405964e4941e4293.64533925.jpg")
img = cv2.imread(img_path)
img = cv2.resize(img,(32,32))
img=np.array(img.astype("float32")/255.0)
img_l.append(img)
img=np.array(img_l)
img_sample = vae.predict(img)
plt.subplot(1,2,1)
plt.imshow(img[0])
plt.subplot(1,2,2)
plt.imshow(img_sample[0])

In [ ]:
img_l=[]
img_path =("../input/someface2/depositphotos.jpg")
img = cv2.imread(img_path)
img = cv2.resize(img,(32,32))
img=np.array(img.astype("float32")/255.0)
img_l.append(img)
img=np.array(img_l)
img_sample = vae.predict(img)
plt.subplot(1,2,1)
plt.imshow(img[0])
plt.subplot(1,2,2)
plt.imshow(img_sample[0])